In [1]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from datetime import datetime,timedelta
from collections import defaultdict

In [2]:
user_index = json.load(open("../data/user_index.json",'r'))
commitment_index = json.load(open("../data/commitment_index.json",'r'))

In [3]:
max_user_id = max([int(i) for i in user_index.keys()])

In [4]:
for k,v in tqdm(user_index.items()):
    user_index[k]['date'] = datetime.strptime(v['date'], '%d %b %Y')

  0%|          | 0/655750 [00:00<?, ?it/s]

In [5]:
id_for_1_jan_2023 = 722621
for k,v in tqdm(user_index.items()):
    if int(k)<id_for_1_jan_2023 and v['date'].year == 2023:
        user_index[k]['date'] = None

  0%|          | 0/655750 [00:00<?, ?it/s]

In [6]:
key_date = user_index[str(max_user_id)]['date']
for i in tqdm(range(max_user_id,id_for_1_jan_2023,-1)):
    if str(i) in user_index.keys():
        if user_index[str(i)]['date']> key_date:
            user_index[str(i)]['date'] = None
        else:
            key_date = user_index[str(i)]['date']

  0%|          | 0/20302 [00:00<?, ?it/s]

In [7]:
last_date = user_index['1']['date']
for i in range(1, max_user_id+1):
    if str(i) in user_index.keys():
        if user_index[str(i)]['date'] == None:
            user_index[str(i)]['date'] = last_date
        else:
            last_date = user_index[str(i)]['date']

In [8]:
for k,v in tqdm(commitment_index.items()):
    commitment_index[k]['start_date'] = datetime.strptime(v['start_date'], '%B %d, %Y')
    commitment_index[k]['end_date'] = datetime.strptime(v['end_date'], '%B %d, %Y')

  0%|          | 0/397456 [00:00<?, ?it/s]

In [9]:
for k,v in tqdm(commitment_index.items()):
    if v['start_date'].year < 2007:
        deta = v['end_date'] - v['start_date']
        commitment_index[k]['start_date'] = user_index[str(v['user_id'])]['date']
        commitment_index[k]['end_date'] = commitment_index[k]['start_date'] + deta

  0%|          | 0/397456 [00:00<?, ?it/s]

## overlap detection

In [10]:
with open('../data/Topic_model_classes.json','r') as fp:
    topic_commitments = json.load(fp)

In [11]:
id_to_class_map = {}
for k,v in tqdm(topic_commitments.items()):
    for id_ in tqdm(v,leave=False):
        id_to_class_map[id_] = k

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/102584 [00:00<?, ?it/s]

  0%|          | 0/14202 [00:00<?, ?it/s]

  0%|          | 0/88659 [00:00<?, ?it/s]

  0%|          | 0/10251 [00:00<?, ?it/s]

...

  0%|          | 0/1761 [00:00<?, ?it/s]

  0%|          | 0/4535 [00:00<?, ?it/s]

  0%|          | 0/1312 [00:00<?, ?it/s]

  0%|          | 0/1582 [00:00<?, ?it/s]

  0%|          | 0/2826 [00:00<?, ?it/s]

In [12]:
len(id_to_class_map)

290008

In [13]:
def find_id_class(cid):
    return id_to_class_map.get(cid,'unidentified')

In [14]:
overlap = defaultdict(list)

In [15]:
for k,v in tqdm(commitment_index.items()):
    for cc in user_index[str(v['user_id'])]['commitments']:
        cc_class = find_id_class(str(cc))
        if str(cc)!=k:
            k_class = find_id_class(k)
            if v['start_date']==commitment_index[str(cc)]['start_date'] and v['end_date']==commitment_index[str(cc)]['end_date']:
                intersection_key = "_".join(set([cc_class,k_class]))
                overlap[intersection_key].append(k)
                overlap[intersection_key].append(str(cc))
            elif v['end_date']<commitment_index[str(cc)]['end_date'] and v['end_date']>commitment_index[str(cc)]['start_date'] and v['start_date']<commitment_index[str(cc)]['start_date']:
                intersection_key = "_".join(set([cc_class,k_class]))
                overlap[intersection_key].append(k)
                overlap[intersection_key].append(str(cc))
            elif v['end_date']==commitment_index[str(cc)]['end_date'] and v['start_date']<commitment_index[str(cc)]['start_date']:
                intersection_key = "_".join(set([cc_class,k_class]))
                overlap[intersection_key].append(k)
                overlap[intersection_key].append(str(cc))
            elif v['start_date']>commitment_index[str(cc)]['start_date'] and v['end_date']<commitment_index[str(cc)]['end_date']:
                intersection_key = "_".join(set([cc_class,k_class]))
                overlap[intersection_key].append(k)
                overlap[intersection_key].append(str(cc))
            elif v['start_date']==commitment_index[str(cc)]['start_date'] and v['end_date']>commitment_index[str(cc)]['end_date']:
                intersection_key = "_".join(set([cc_class,k_class]))
                overlap[intersection_key].append(k)
                overlap[intersection_key].append(str(cc))
            elif v['start_date']>commitment_index[str(cc)]['start_date'] and v['start_date']<commitment_index[str(cc)]['end_date'] and v['end_date']>commitment_index[str(cc)]['end_date']:
                intersection_key = "_".join(set([cc_class,k_class]))
                overlap[intersection_key].append(k)
                overlap[intersection_key].append(str(cc))

  0%|          | 0/397456 [00:00<?, ?it/s]

In [16]:
len(overlap.keys())

197

In [17]:
len(set([j for k,v in overlap.items() for j in v]))

164489

In [18]:
with open('../data/Topic_model_overlap_commitments.json','r') as fp:
    multi_commitments = json.load(fp)

In [19]:
for k,v in tqdm(multi_commitments.items()):
    intersection_key = "_".join(set(k.split("_")))
    overlap[intersection_key].extend(v)

  0%|          | 0/136 [00:00<?, ?it/s]

In [20]:
len(overlap.keys())

241

In [21]:
len(set([j for k,v in overlap.items() for j in v]))

167511

In [27]:

167511/len(commitment_index)

0.4214579727064128

In [22]:
counts = []
for k,v in overlap.items():
    counts.append([k,len(set(v))])

In [23]:
pd.DataFrame(counts, columns=['Intersection','Count']).sort_values("Count",ascending=False).head(50)

,Intersection,Count
8,unidentified,39994
0,exercise_unidentified,31160
3,exercise_weight,28286
2,weight_unidentified,19169
18,study_unidentified,10278
9,exercise,9615
7,weight,9067
10,eat_unidentified,8114
15,read_unidentified,7500
11,exercise_eat,7361


In [24]:
with open("../data/Overlap_commitments.json",'w') as fp:
    json.dump(overlap,fp,indent=4)